In [36]:
import requests
import pandas as pd
import time
import pandas as pd
import numpy as np
import requests
import time
#import Levenshtein as lev
#from fuzzywuzzy import fuzz
import re

pd.set_option('display.max_colwidth', None)

In [37]:
def poll_job(s, redash_url, job):
    # TODO: add timeout
    while job['status'] not in (3,4):
        response = s.get('{}/api/jobs/{}'.format(redash_url, job['id']))
        job = response.json()['job']
        time.sleep(1)
    if job['status'] == 3:
        return job['query_result_id']    
    return None

In [38]:
def get_fresh_query_result(redash_url, query_id, api_key):
    s = requests.Session()
    s.headers.update({'Authorization': 'Key {}'.format(api_key)})
    response = s.post('{}/api/queries/{}/refresh'.format(redash_url, query_id))
    print(response.text)
    if response.status_code != 200:
        raise Exception('Refresh failed.')
    result_id = poll_job(s, redash_url, response.json()['job'])
    if result_id:
        response = s.get('{}/api/queries/{}/results/{}.json'.format(redash_url, query_id, result_id))
        if response.status_code != 200:
            raise Exception('Failed getting results.')
    else:
        raise Exception('Query execution failed.')
    return response.json()['query_result']['data']['rows']

In [39]:
#reading crawls data from redash
api_key_gi = '7k07SN8ZqbBGbkBhy4vk0qeDP4NGe8meXPJmgwNm'
vendor_properties = pd.DataFrame.from_dict(get_fresh_query_result('https://hotels-redash.mmt.live/',12987,api_key_gi))
print("Vendor Properties- ",vendor_properties.shape)

{"job": {"status": 1, "error": "", "id": "d4291bf0-31be-47cf-adbc-c0dfb89b17f5", "query_result_id": null, "updated_at": 0}}
Vendor Properties-  (110, 2)


In [40]:
#vendor_properties

In [41]:
vendor_properties

image_info  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [42]:
image_info=vendor_properties

In [43]:
df = pd.DataFrame(image_info)

In [28]:
output_df

,URL,MMT Tags
0,https://gos3.ibcdn.com/af85276f-30a8-4eae-a1c4-0a5cba0d6a5a.png,Swimming Pool
1,https://gos3.ibcdn.com/0359bf1b-fb9b-4256-aa23-f5a3b83320fe.jpg,Balcony
2,https://gos3.ibcdn.com/9f531dcecea111ebb8720242ac11000d.jfif,Facade
3,https://gos3.ibcdn.com/42a45b5ecea311eb9fb80242ac11000d.jpg,Facade
4,https://gos3.ibcdn.com/e7da15ec5c7e11ebac1e0242ac110003.jpg,Outdoors
...,...,...
65,https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202101291732137638-ea095982-302e-4612-93dc-9d9bf06df2f1.jpg,Reception
66,https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202101291732137638-99cce16f-0884-4edb-b80f-2291aee86489.jpg,Others
67,https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202101291732137638-d1a5e7b8-4df7-4fd3-80af-d5547d9094eb.jpg,Facade
68,https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202101291732137638-6bb1a261-96bb-4dd6-b0cc-5480310e6a97.jpg,Facade


In [ ]:
#D:\downloads\ImageComp\PoC

In [47]:
#df = pd.DataFrame(data)

# Extract and process the 'image_info' column
if 'image_info' in df.columns:
    # Extract the list of dictionaries
    all_image_info = df['image_info'][0]  # Assuming the data is in the first row
    if isinstance(all_image_info, list):
        # Process the extracted list
        result = [
            {
                'URL': item.get('url', 'N/A'),
                'MMT Tags': ', '.join(item.get('mmt_tags', [])),
                'MMT IDs': ', '.join(map(str, item.get('mmt_htl_id', [])))
            }
            for item in all_image_info
        ]
        # Convert to DataFrame
        output_df = pd.DataFrame(result)
        print(output_df)
        # Save to Excel
        output_df.to_excel('image_urls_with_mmt_tags_and_ids.xlsx', index=False)
    else:
        print("The 'image_info' column does not contain a list as expected.")
else:
    print("The DataFrame does not contain an 'image_info' column.")

                                                                                                                      URL  \
0   https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/room-imgs/202109151802278551-2312-b715c87a161911eca38a0a58a9feac02.jpg   
1         https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-eea89232161811eca5570a58a9feac02.jpg   
2         https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-8f51a0bc161811ecb6fd0a58a9feac02.jpg   
3         https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-48b37662161811ecb5760a58a9feac02.jpg   
4         https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-6860cf14161811ecb6fd0a58a9feac02.jpg   
5         https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-c1b576dc161811ecbf440a58a9feac02.jpg   
6         https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-c6764e7c161711ec87cb0a58a9feac02.jpg   


In [50]:
# Process 'image_info' column
output_data = []
for idx, row in df.iterrows():
    image_info_list = row['image_info']
    mmt_htl_id = row['mmt_htl_id']
    
    if isinstance(image_info_list, list):  # Ensure it's a list
        for item in image_info_list:
            output_data.append({
                'MMT Hotel ID': mmt_htl_id,
                'URL': item.get('url', 'N/A'),
                'MMT Tags': ', '.join(item.get('mmt_tags', [])),
                'MMT IDs': ', '.join(map(str, item.get('mmtids', []))),
                'Media Tag': item.get('media_tag', 'N/A'),
                'Image Title': item.get('image_title', 'N/A')
            })

# Convert to DataFrame
output_df = pd.DataFrame(output_data)

# Save to Excel
output_df.to_excel('D:\downloads\ImageComp\PoC\image_urls_with_mmt_htl_id.xlsx', index=False)

print("D:\downloads\ImageComp\PoC added")
# Display the output DataFrame
print(output_df)

D:\downloads\ImageComp\PoC added
            MMT Hotel ID  \
0     202109151802278551   
1     202109151802278551   
2     202109151802278551   
3     202109151802278551   
4     202109151802278551   
...                  ...   
6162  201709081331037533   
6163  201709081331037533   
6164  201709081331037533   
6165  201709081331037533   
6166  201709081331037533   

                                                                                                                        URL  \
0     https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/room-imgs/202109151802278551-2312-b715c87a161911eca38a0a58a9feac02.jpg   
1           https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-eea89232161811eca5570a58a9feac02.jpg   
2           https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-8f51a0bc161811ecb6fd0a58a9feac02.jpg   
3           https://r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202109151802278551-48b37662161811ecb5760a58a9feac02.jp